In [ ]:
import pandas as pd
import numpy as np

# change the display width to see all columns
pd.set_option('display.width', 1000)

games_df = pd.read_csv('./data/games.csv', low_memory=False)
games_details_df = pd.read_csv('./data/games_details.csv', low_memory=False)
# players_df = pd.read_csv('./data/players.csv')
ranking_df = pd.read_csv('./data/ranking.csv')
teams_df = pd.read_csv('./data/teams.csv')

In [ ]:
games_df = games_df.drop(columns=['GAME_STATUS_TEXT'])
initial_elo = 1500
k_factor = 20

# create a dictionary to store the elo of each team
elo_dict = {}

def expected_outcome(home_elo, away_elo):
    return 1 / (1 + 10 ** ((away_elo - home_elo) / 400))

def update_elo(home_elo, away_elo, home_win):
    expected_win = expected_outcome(home_elo, away_elo)
    actual_home_win = 1 if home_win else 0
    new_home_elo = home_elo + k_factor * (actual_home_win - expected_win)
    new_away_elo = away_elo + k_factor * ((1-actual_home_win) - (1-expected_win))
    return new_home_elo, new_away_elo


In [ ]:
# Create margin of victory column
games_df["MOV"] = games_df["PTS_home"] - games_df["PTS_away"]
close_games = games_df[(games_df["MOV"] > -5) & (games_df["MOV"] < 5)]
print('Number of close games:', close_games.shape[0])

close_game_prob = close_games["MOV"].mean()*100
print('Probability of a close game:', close_game_prob)


In [ ]:
# Create high scoring game column
games_df["total_score"] = games_df["PTS_home"] + games_df["PTS_away"]
high_scoring_games = games_df[games_df["total_score"] > 220]
print('Number of high scoring games:', high_scoring_games.shape[0])

high_scoring_game_prob = len(high_scoring_games) / len(games_df)*100
print('Probability of a high scoring game:', high_scoring_game_prob)

Below is a rough outline of the final recommender system that I will be building in this notebook. It takes in the schedule for upcoming games, and outputs the recommended games that will be the closest and most exciting to watch based on the historical data from NBA games.

In [ ]:
# from sklearn.neighbors import NearestNeighbors
# from sklearn.metrics import f1_score

# # Load the trained model
# model = NearestNeighbors(n_neighbors=5, metric='cosine')
# model.load('nba_game_recommender.pkl')

# # Load the schedule for the upcoming night's games
# schedule = pd.read_csv('nba_schedule.csv')

# # Extract relevant features for each game
# features = []
# for index, row in schedule.iterrows():
#     game_id = row['Game_ID']
#     home_team = row['Home_Team']
#     away_team = row['Away_Team']
#     # Extract features from historical data
#     feature_vector = get_features(game_id, home_team, away_team)
#     features.append(feature_vector)

# # Use the model to predict the most exciting game(s) to watch
# distances, indices = model.kneighbors(features)
# recommended_games = []
# for i, dist in enumerate(distances):
#     if dist < 0.5:  # Arbitrarily set a threshold